## Root Finding Algorithms for Implied Volatility (Vanilla Options)

Deriving the Implied Volatilities from market prices is an inverse problem. Instead of finding the inverted form of the put or call price formula, if an invertible form even exists, we can instead turn to two very common root finding methods.

## Bisection Method

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm


In [2]:
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

In [3]:
# We shall arbitrarily give the inputs
S = 140
K = 150
T = 3
r = 0.05
IV_Known= 0.3

Call_Price = BS_CALL(S, K, T, r, IV_Known)
print(Call_Price)

33.355467060212774


We now want to find the unknown root (IV) that will give the current call option price

BISECTION METHOD
The bisection method can only be used for finding roots, that is f(r)=0 and requires
the Intermediate Value Theorem to be True. Thus, to make the bisection method work, we will
shift BS_CALL(IV, .) by - Price to make it zero when the right IV is found



In [5]:
a = 0.01 #Lower guess for IV
b = 0.9 # Upper Guess for IV
acc = 0.001 # accuracy / tolerance for between actual IV

def bisect(BS_CALL, a, b, acc):
    

    if (BS_CALL(S, K, T, r, a)-Call_Price)*(BS_CALL(S, K, T, r, b)-Call_Price) > 0:
   
      
        raise Exception("No root within the interval")
        
    m = (a+b)/2 
      

    if np.abs(BS_CALL(S, K, T, r, m) -Call_Price  ) < acc:
        return m ,print("IV of BS_CALL at the approx root using Bisection Method =", m)
    
    
    elif np.sign(BS_CALL(S, K, T, r, a) -Call_Price) == np.sign(BS_CALL(S, K, T, r, m)-Call_Price):
       
        return bisect(BS_CALL, m, b, acc)
    
    elif np.sign(BS_CALL(S, K, T, r, b)-Call_Price) == np.sign(BS_CALL(S, K, T, r, m)-Call_Price):
 
        return bisect(BS_CALL, a, m, acc)

approx_IV = bisect(BS_CALL, a, b, acc)
print(approx_IV)

IV of BS_CALL at the approx root using Bisection Method = 0.2999942016601562
(0.2999942016601562, None)


## Newton's Method

For Newtons method to work, we will require the derivative of the BS_CALL with
respect to volatility which is also known as vega. Note, Vega for a Put or Call is the same by Put Call parity

In [6]:
def Vega_BS_Call(S, K, T, r, sigma):
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    return S*np.sqrt(T) * norm.pdf(d1)


Xo = 0.4 #Inital guess for IV         
tol = 0.001
            

def newt(BS_CALL, Vega_BS_Call, Xo, tol):
    

    if abs(BS_CALL(S, K, T, r, Xo) - Call_Price) < tol:
        return Xo, print("IV of BS_CALL at the approx root using Newton Method =", Xo)
    

    else:
        return newt(BS_CALL, Vega_BS_Call , Xo - (BS_CALL(S, K, T, r, Xo)-Call_Price)/Vega_BS_Call(S, K, T, r, Xo), tol)
 

approx_iv_newt = newt(BS_CALL, Vega_BS_Call, Xo, tol)
print(approx_iv_newt)

IV of BS_CALL at the approx root using Newton Method = 0.29999990466614934
(0.29999990466614934, None)
